In [2]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import os
import re
import pickle
from tqdm.notebook import tqdm
from difflib import SequenceMatcher
import openai

# Load local modules
from module_text_blocks import split_text_into_blocks, clean_text_blocks

# load credentials for OpenAI API
from credentials_openai import *
openai.api_key = openai_api_key

In [3]:
import tiktoken

# define the number of tokens in the prompt
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [4]:
def split_text_by_chars(text, num_chars):
    """Split the input text every num_chars characters."""
    return [text[i:i+num_chars] for i in range(0, len(text), num_chars)]

In [5]:
def summarize_long_text_blocks(text_blocks):

    for heading, text in text_blocks.items():
        
        text_length = len(text.split(' '))                           # number of words in the text block
        tokens_number = num_tokens_from_string(text, "cl100k_base")  # number of tokens in the text block
        chars_number = len(text)                                     # number of characters in the text block
                        
        summarization_blocks = [text]                                # list of text blocks to summarize
        responses = []

        # if the block contains over 750 words, summarize it
        if text_length > 750:

            # if the block is exceeding the token limit, split it into multiple blocks
            if tokens_number > 3500:

                text_split_threshold = int(chars_number / (tokens_number / 2500))
                summarization_blocks = split_text_by_chars(text, text_split_threshold)
                
            for summarization_block in summarization_blocks:

                completion = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[
                    {"role": "user", "content": "Please effectively summarize the following text: " + summarization_block}
                    ],
                    temperature=0.3,
                    max_tokens=500
                )                
                # top_p=1, frequency_penalty=0, presence_penalty=0, stop=["\n"]
                # add the summarized text to the list of responses
                responses.append(completion.choices[0].message.content)

            # join the responses into a single text block
            text_blocks[heading] = ' '.join(responses)
    
    return text_blocks

In [6]:
def openai_sentiment_analysis(final_prompt):

    # run the request for ChatGPT
    fine_tune_messages = {"role": "system", "content":
                    "You are a helpful financial assistant who is expert in evaluating sentiment scores for financial statements \
                You give precise answers to questions \
                the quality of your answers is highly important, you never hallucinate answers - only \
                answering based on your knowledge. Where the answer requires creative thought you engage \
                in reflective internal dialogue to ascertain the best answer"
    }

    user_content = "Calculate the total polarity and subjectivity scores on the strict range -1 to 1 (-1 means perfectly negative; 1 means perfectly positive): "


    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        fine_tune_messages,
        {"role": "user", "content": user_content + final_prompt}
    ]
    )

    return completion.choices[0].message.content

In [14]:
def openai_sentiment_analysis(final_prompt):

    # run the request for ChatGPT
    fine_tune_messages = {"role": "system", "content":
                    "You are a helpful financial assistant who is expert in evaluating sentiment scores for financial statements \
                You give precise answers to questions \
                the quality of your answers is highly important, you never hallucinate answers - only \
                answering based on your knowledge. Where the answer requires creative thought you engage \
                in reflective internal dialogue to ascertain the best answer"
    }

    user_content = "The overall polarity and subjectivity scores on the strict range (very negative, moderately negative, slightly negative, neutral, slightly positive, moderately positive, very positive) for the text: "

    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        fine_tune_messages,
        {"role": "user", "content": user_content + final_prompt}
    ],
    temperature=0.0,
    max_tokens=1500
    )

    return completion.choices[0].message.content

In [8]:
# Load pdf text and headings from the pickle file
pdf_texts = pickle.load(open("pdf_texts.pkl", "rb"))
pdf_headings = pickle.load(open("pdf_headings.pkl", "rb"))
pdf_headings_context = pickle.load(open("pdf_headings_context.pkl", "rb"))

In [15]:
# store openai responses in a dictionary
openai_responses = {}
# pdf_lists = ["FINAL-Q1-19-Shareholder-Letter",
#             "FINAL-Q420-Shareholder-Letter",
#             "Q2-19-Shareholder-Letter-FINAL"]

pdf_lists = ["Q1_15_Earnings_Letter_final_tables"]

for pdf_name in tqdm(pdf_lists):

    try:

        text = pdf_texts[pdf_name]
        headings = pdf_headings[pdf_name]
        headings_context = pdf_headings_context[pdf_name]

        # split the text into blocks based on the headings
        text_blocks = split_text_into_blocks(text, headings, headings_context)
        
        # clean the text blocks
        text_blocks = clean_text_blocks(text_blocks)

        # if there are headings, add headings to the text blocks and summarize if needed
        if len(headings) > 0:
            
            # print the original length of the text blocks
            print("Original length of blocks for " + pdf_name + ":")
            for heading, text in text_blocks.items():
                print(len(text.split(" ")), end=" ")
            print(" ")

            # summarize the text blocks
            text_blocks = summarize_long_text_blocks(text_blocks)

            # print the length of the text blocks after summarization
            print("Updated length of blocks for " + pdf_name + ":")
            for heading, text in text_blocks.items():
                print(len(text.split(" ")), end=" ")
            print(" ")

            # Create a final prompt
            final_prompt = ''
            for heading, text in text_blocks.items():
                if num_tokens_from_string(final_prompt + text, "cl100k_base") >= 3750:
                    break
                final_prompt += heading + ': ' + text + " "

        # if there are no headings  
        else:
            
            tokens_number = num_tokens_from_string(text, "cl100k_base")  # number of tokens in the text block
            chars_number = len(text)                                     # number of characters in the text block
            
            # if the block is exceeding the token limit, cut it
            if tokens_number > 3500:

                text_split_threshold = int(chars_number / (tokens_number / 2500))
                text = text[:text_split_threshold]

            final_prompt = text
            

        # Perform openAI sentiment analysis
        response = openai_sentiment_analysis(final_prompt)
        openai_responses[pdf_name] = response

        # response = openai_sentiment_analysis_finetune(final_prompt, response)
        # fine_tuned_responses[pdf_name] = response

        # break

    except Exception as e:

        print(f"Exception occurred in file {pdf_name}")
        print(f"Exception message: {str(e)}")

        continue

        # break

  0%|          | 0/1 [00:00<?, ?it/s]

Original length of blocks for Q1_15_Earnings_Letter_final_tables:
119 561 244 81 138 184 125 55 97 36 27 53 1401  
Updated length of blocks for Q1_15_Earnings_Letter_final_tables:
119 561 244 81 138 184 125 55 97 36 27 53 65  


In [17]:
text_blocks

{'Document_intro': "We achieved several major milestones in Q1: surpassing 40 million members in the US; 20 million internationally; and 60 million in total. Our original series, documentaries and comedy specials are being enthusiastically received, and member engagement is at an all-time high. Members streamed 10 billion hours in Q1, more evidence that consumers around the world are embracing the Internet TV revolution. *Q1'15 EPS would have been $0.77 excluding a F/X lloss. Q4'14 Net Income/EPS includes a $39m / $0.63 benefit from a tax accrual release related to resolution of tax audit **Corresponds to our total known streaming content oobligations as defined in our financial statements and related notes in our most recently filed SEC Form 10-K 1 2 ",
 'Q1 Results and Q2 Forecast': 'We added a record 4.9 million new members globally in Q1, against our forecast of 4.1 million and prior year of 4.0 million, bringing our total global streaming membership to 62.3 million. In the US, we 

In [16]:
openai_responses

{'Q1_15_Earnings_Letter_final_tables': 'The overall polarity and subjectivity scores for the text are as follows:\n\nPolarity:\n- Very negative: 0\n- Moderately negative: 0\n- Slightly negative: 0\n- Neutral: 0\n- Slightly positive: 0\n- Moderately positive: 0\n- Very positive: 0\n\nSubjectivity:\n- Objective: 100%\n- Subjective: 0%'}

In [12]:
# Save openai responses to pickle file
with open("Src/openai_responses_test_v2.pkl", "wb") as f:
    pickle.dump(openai_responses, f)

In [17]:
# Load openai responses from the pickle file
openai_responses = pickle.load(open("Src/openai_responses_test.pkl", "rb"))

In [13]:
openai_responses_test = pd.DataFrame(list(openai_responses.items()), columns=['pdf_name', 'response'])

In [17]:
openai_responses_test['response'] = openai_responses_test['response'].str.replace("The overall polarity and subjectivity scores for the text are as follows:", "")

In [18]:
openai_responses_test

,pdf_name,response
0,COMBINED-Q4-17-Shareholder-Letter-FINAL,The overall polarity and subjectivity scores f...
1,FINAL-Q1-18-Shareholder-Letter,The overall polarity and subjectivity scores f...
2,FINAL-Q1-19-Shareholder-Letter,The overall polarity and subjectivity scores f...
3,FINAL-Q1-20-Shareholder-Letter,The overall polarity and subjectivity scores f...
4,FINAL-Q1-21-Shareholder-Letter,The overall polarity and subjectivity scores f...
5,FINAL-Q1-22-Shareholder-Letter,The overall polarity and subjectivity scores f...
6,Final-Q1-23-Shareholder-Letter,The overall polarity and subjectivity scores f...
7,FINAL-Q2-18-Shareholder-Letter,The overall polarity and subjectivity scores f...
8,FINAL-Q2-20-Shareholder-Letter-V3-with-Tables,The overall polarity score for the text is sli...
9,FINAL-Q2-21-Shareholder-Letter,\n\nPolarity: Slightly positive\nSubjectivity:...


In [14]:
# export OpenAI responses to excel
openai_responses_test.to_excel('Src/openai_responses_test_v2.xlsx')

In [72]:
# clean the text of the responses
for key in openai_responses:
        openai_responses[key] = re.sub('\n', ' ', openai_responses[key])
        openai_responses[key] = re.sub(' +', ' ', openai_responses[key])

In [98]:
# Extract polarity and subjectivity scores from OpenAI responses
polarity_scores = {}
subjectivity_scores = {}


for document, text in openai_responses.items():

    words = text.split(" ")
    for word in words:
        if word.lower() == "polarity":
            polarity_scores[document] = words[words.index(word):words.index(word) + 4]
        elif word.lower() == "subjectivity":
            subjectivity_scores[document] = words[words.index(word):words.index(word) + 4]

In [102]:
# Extract digits from polarity_scores dictionary
for document, words in polarity_scores.items():

    for word in words:
        # if word is number using regex
        if re.search("^[0-9]", word):
            word = word.replace(",", "")
            polarity_scores[document] = float(word)

In [104]:
# Extract digits from subjectivity scores dictionary
for document, words in subjectivity_scores.items():

    for word in words:
        # if word is number using regex
        if re.search("^[0-9]", word):
            word = word.replace(",", "")
            subjectivity_scores[document] = float(word)

In [107]:
# filter out keys values of which are lists
polarity_scores = {k: v for k, v in polarity_scores.items() if type(v) != list}
subjectivity_scores = {k: v for k, v in subjectivity_scores.items() if type(v) != list}

In [124]:
df = pd.DataFrame(list(pdf_texts.items()), columns=['pdf_name', 'text'])
polarity_df = pd.DataFrame(list(polarity_scores.items()), columns=['pdf_name', 'polarity'])
subjectivity_df = pd.DataFrame(list(subjectivity_scores.items()), columns=['pdf_name', 'subjectivity'])

# join df and polarity_df on pdf_name column
df = df.join(polarity_df.set_index("pdf_name"), on="pdf_name")
df = df.join(subjectivity_df.set_index("pdf_name"), on="pdf_name")

In [127]:
# export df
df.to_excel('Score/OpenAI_scores.xlsx', index=False)